In [ ]:
import argparse
import geopandas
import gzip
import io
import json
from matplotlib.collections import LineCollection
import pandas as pd
import plotly
import plotly.express as px
from pprint import pprint
import requests
import main
import sierra_leone

# %load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
chief = main.load_chiefdom_geopandas()
chief_geojson = main.load_chiefdom_geojson()
adj = main.load_chiefdom_adjacency_data()
trigger_na, trigger_ave, trigger_other, follow_up, follow_up_other, codebook = main.load_all_paper_data()
digital = main.load_digital_data()


In [ ]:
conf = main.load_cases_confirmed()
conf

In [ ]:
susp = main.load_cases_suspected()
susp

In [ ]:
def animate_map_1(chief, chief_geojson):

    # If your shapefile has coordinates in UTM format,
    # and you want to transform them into WGS84 (Lat / Lon format), you can do this:
    # geodf = geodf.to_crs({'init': 'epsg:4326'})
    print(chief.crs)

    print(plotly.__version__)
    # Plot Chiefdoms, colored by district
    fig = px.choropleth(
        chief,
        geojson=chief_geojson,
    #     color='Shape_Area',
        color='admin2Name',
        locations="OBJECTID",
        featureidkey="properties.OBJECTID",
        )
    # go.Figure()
    fig.update_geos(fitbounds="locations", visible=False)
    # fig.update_geos(lataxis_showgrid=True, lonaxis_showgrid=True)
    fig.update_layout(mapbox_style="white-bg")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

def animate_map_over_time(chief)
# animate_map_1(chief, chief_geojson)
animate_map_over_time(chief, chief_geojson)



In [ ]:
chief.geometry.centroid


In [ ]:

def plot_chiefdom_adjacency(chief, adj):
    # Create lines for each adjacent pair of chiefdoms in the format that matplotlib likes
    df = chief.copy()
    df['centroid'] = df.geometry.centroid
    df = df.loc[:, ['OBJECTID', 'centroid']]
    # add centroids to adj
    cents = (adj.merge(df, how='inner', left_on='OBJECTID1', right_on='OBJECTID').drop(columns=['OBJECTID'])
             .merge(df, how='inner', left_on='OBJECTID2', right_on='OBJECTID').drop(columns=['OBJECTID'])
             .rename(columns={'centroid_x': 'centroid1', 'centroid_y': 'centroid2'}))
    cents = cents.loc[cents['OBJECTID1'] != cents['OBJECTID2']] 
    # one line per pair of chiefdoms in adj, 2 points in each line, (x,y) pair for each point.
    segments = np.asarray(list(cents.apply(lambda s: ((s['centroid1'].x, s['centroid1'].y), (s['centroid2'].x, s['centroid2'].y)), axis=1)
                     .values))
    lc = LineCollection(segments) #, linewidths=(0.5, 1, 1.5, 2),
    #                                colors=colors, linestyle='solid')

    # plot map of sierra leone chiefdoms, the centroid of each chiefdom, and lines connecting adjacent centroids.
    # https://geopandas.org/mapping.html#maps-with-layers

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_aspect('equal')
    chief.plot(ax=ax, color='orange', alpha=0.3, edgecolor='black')
    for i in range(segments.shape[0]):
        if i > 10:
            pass
            # break

        ax.plot(segments[i, :, 0], segments[i, :, 1], color='b')

    chief.geometry.centroid.plot(ax=ax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('The Sierra Leone Chiefdom Adjacency Network')
    plt.show()
    # ax.add_collection(lc)
    # df.apply(lambda s: ((s['centroid1'].x, s['centroid1'].y), (s['centroid2'].x, s['centroid2'].y)) axis=1)

    # xs = []
    # ys = []
    # for i 
    # i = 100
    # j = 1
    # cent1 = chief.geometry.centroid[i]
    # cent2 = chief.geometry.centroid[j]
    # x1, y1 = cent1.x, cent1.y
    # x2, y2 = cent2.x, cent2.y
    # ax.plot([x1, x2], [y1, y2])


In [ ]:
plot_chiefdom_adjacency(chief, adj)
